In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

plt.style.use(
    "https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle"
)

In [2]:
# Load the data
data_raw = pd.read_excel(r'C:\Users\rrenard\OneDrive - Arkus\Desktop\UHeston_test.xlsx')

In [3]:
df = data_raw.copy()

In [4]:

df['return'] = np.log(df['SPX'] / df['SPX'].shift(1)) *100
df_mean = df['return'].mean()
df['residual'] = df['return'] - df_mean
df['realised variance'] = df['residual']**2
df.dropna(inplace=True)
start_variance = np.var(df['residual'])
kappa = 1
long_term_variance = start_variance
df['expected variance'] = df['realised variance'].shift(1) + kappa * (long_term_variance - df['realised variance'].shift(1).astype(float))  # type: ignore
df['expected variance'].iloc[0] = start_variance
volatility_of_volatility = np.std(df['realised variance'])
rho = 0

C:\Users\rrenard\AppData\Local\Temp\ipykernel_23752\2224834979.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['expected variance'].iloc[0] = start_variance


In [17]:
df_mean, volatility_of_volatility

(0.0945126849779724, 1.1152520175848961)

In [6]:
df

,Date,SPX,return,residual,realised variance,expected variance
1,2021-01-04,3700.649902,-1.486476,-1.580989,2.499526,0.678995
2,2021-01-05,3726.860107,0.705763,0.611250,0.373627,0.678995
3,2021-01-06,3748.139893,0.569360,0.474848,0.225480,0.678995
4,2021-01-07,3803.790039,1.473826,1.379313,1.902505,0.678995
5,2021-01-08,3824.679932,0.547684,0.453171,0.205364,0.678995
...,...,...,...,...,...,...
248,2021-12-27,4791.189941,1.374405,1.279892,1.638125,0.678995
249,2021-12-28,4786.350098,-0.101067,-0.195579,0.038251,0.678995
250,2021-12-29,4793.060059,0.140091,0.045579,0.002077,0.678995
251,2021-12-30,4778.729980,-0.299423,-0.393936,0.155186,0.678995


In [13]:
# function for the log likelihood
def log_likelihood_computation(data, params):
    log_likelihood_part1 =  1 / (2 * np.pi * np.sqrt(data['expected variance']) * params['sigma']['x0'] * np.sqrt(1-params['rho']['x0']**2))
    log_likelihood_part2 = np.exp(-((((data['residual']/np.sqrt(data['expected variance']))**2) 
                                    - 2 * params['rho']['x0'] * (data['residual']/np.sqrt(data['expected variance'])) * 
                                    (((data['realised variance'] - data['expected variance'])/params['sigma']['x0'])) + 
                                    (((data['realised variance'] - data['expected variance'])/params['sigma']['x0'])**2)) / (2 * (1 - params['rho']['x0']**2))))
    data['log likelihood'] = np.log(log_likelihood_part1 * log_likelihood_part2)
    return data

In [14]:
params = {'v0': start_variance, 'kappa': kappa, 'theta': long_term_variance, 'sigma': volatility_of_volatility, 'rho': rho}
params = pd.DataFrame(params, index=['x0'])
params['kappa']['x0']

1

In [15]:
df_test = log_likelihood_computation(df, params)

In [16]:
df_test

,Date,SPX,return,residual,realised variance,expected variance,log likelihood
1,2021-01-04,3700.649902,-1.486476,-1.580989,2.499526,0.678995,-4.926350
2,2021-01-05,3726.860107,0.705763,0.611250,0.373627,0.678995,-2.066005
3,2021-01-06,3748.139893,0.569360,0.474848,0.225480,0.678995,-2.002107
4,2021-01-07,3803.790039,1.473826,1.379313,1.902505,0.678995,-3.756141
5,2021-01-08,3824.679932,0.547684,0.453171,0.205364,0.678995,-1.994792
...,...,...,...,...,...,...,...
248,2021-12-27,4791.189941,1.374405,1.279892,1.638125,0.678995,-3.329484
249,2021-12-28,4786.350098,-0.101067,-0.195579,0.038251,0.678995,-1.946595
250,2021-12-29,4793.060059,0.140091,0.045579,0.002077,0.678995,-1.939119
251,2021-12-30,4778.729980,-0.299423,-0.393936,0.155186,0.678995,-1.977961


In [18]:
np.sum(df_test['log likelihood'])

-693.8533993499839